In [ ]:
%matplotlib inline


NST via image-based optimization
================================



imports



In [ ]:
from collections import OrderedDict
import torch
from torch import optim
from pystiche.image import show_image, write_image
from pystiche.enc import vgg19_encoder
from pystiche.ops import MSEEncodingOperator, GramOperator, MultiLayerEncodingOperator
from pystiche.loss import MultiOperatorLoss
from utils import demo_images

Load the encoder used to create the feature maps for the NST



In [ ]:
multi_layer_encoder = vgg19_encoder()

Create the content loss



In [ ]:
content_layer = "relu_4_2"
content_encoder = multi_layer_encoder[content_layer]
content_weight = 1e0
content_loss = MSEEncodingOperator(content_encoder, score_weight=content_weight)

Create the style loss



In [ ]:
style_layers = ("relu_1_1", "relu_2_1", "relu_3_1", "relu_4_1", "relu_5_1")
style_weight = 1e4


def get_style_op(encoder, layer_weight):
    return GramOperator(encoder, score_weight=layer_weight)


style_loss = MultiLayerEncodingOperator(
    multi_layer_encoder, style_layers, get_style_op, score_weight=style_weight,
)

Combine the content and style loss into the optimization criterion



In [ ]:
criterion = MultiOperatorLoss(
    OrderedDict([("content_loss", content_loss), ("style_loss", style_loss)])
)

Make this demo device-agnostic



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = criterion.to(device)

load the content and style images and transfer them to the selected device
the images are resized, since the stylization is memory intensive



In [ ]:
size = 500
images = demo_images()
content_image = images["dancing"].read(size=size, device=device)
style_image = images["picasso"].read(size=size, device=device)
show_image(content_image)
show_image(style_image)

Set the target images for the content and style loss



In [ ]:
content_loss.set_target_image(content_image)
style_loss.set_target_image(style_image)

Set the starting point of the stylization to the content image. If you want
to start from a white noise image instead, uncomment the line below



In [ ]:
input_image = content_image.clone()

<div class="alert alert-info"><h4>Note</h4><p>To avoid boilerplate code, you can achieve the same behavior with
  :func:`~pystiche.misc.misc.get_input_image`::

    from pystiche.misc import get_input_image

    starting_point = "content"
    input_image = get_input_image(starting_point, content_image=content_image)</p></div>

<div class="alert alert-info"><h4>Note</h4><p>If you want to start the stylization from a white noise image instead, you
  can use::

    input_image = torch.rand_like(content_image)

  or::

    starting_point = "random"
    input_image = get_input_image(starting_point, content_image=content_image)</p></div>



Create the optimizer that performs the stylization



In [ ]:
optimizer = optim.LBFGS([input_image.requires_grad_(True)], lr=1.0, max_iter=1)

<div class="alert alert-info"><h4>Note</h4><p>To avoid boilerplate code, you can achieve the same behavior with
  :func:`~pystiche.optim.optim.default_image_optimizer`::

    from pystiche.optim import default_image_optimizer

    optimizer = default_image_optimizer(input_image)</p></div>



Run the stylization



In [ ]:
# num_steps = 500
# for step in range(1, num_steps + 1):
#
#     def closure():
#         optimizer.zero_grad()
#         loss = criterion(input_image)
#         loss.backward()
#
#         if step % 50 == 0:
#             print(loss.aggregate(1))
#             print("-" * 80)
#
#         return loss
#
#     optimizer.step(closure)

<div class="alert alert-info"><h4>Note</h4><p>To avoid boilerplate code, you can achieve the same behavior with
  :func:`~pystiche.optim.optim.default_image_optim_loop`::

    from pystiche.optim import default_image_optim_loop

    default_image_optim_loop(
        input_image, criterion, optimizer=optimizer, num_steps=num_steps
    )</p></div>



Show the stylization result



In [ ]:
show_image(input_image)